<a href="https://colab.research.google.com/github/nadia-data/nadia-data/blob/main/Test_technique_EL_HANINE_Nadia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### load and describe the data

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
data_train=pd.read_csv('hour.csv') # reading our training data
data_train.head()# visualizing the first five rows of our data 

In [ ]:
data_train.shape # checking Shape of the data

In [ ]:
 data_train.describe() # seeing statistics of our data 

In [ ]:
data_train.isnull().sum() # checking missing values in our data

In [ ]:
data_train.info()  # checking missing values in our data and the columns type of the data 

In [ ]:
data_train.insert(4,"day", pd.DatetimeIndex(data_train['dteday']).day, allow_duplicates=False) # add the day to our data from the date 

In [ ]:
data_train.drop(['dteday','instant'], axis = 1, inplace = True)# get rid of columns that are not necessary for our prediction
data_train.head() #visualize our data again to check if the day has been added and the columns has been removed

#make graphic


In [ ]:
data_graphic=data_train.copy() #taking a copy of the data to build useful graphs
data_graphic.head()


In [ ]:
data_graphic['season'] = data_graphic.season.map({1:'Winter', 2:'Spring', 3:'Summer', 4:'Fall'}) #add the name of each season 
data_graphic['weathersit'] =data_graphic.weathersit.map({1:'Clear', 2:'Mist', 3:'Light rain', 4:'Heavy rain'})#add the name for each weather 

In [ ]:
data_graphic.head() # 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (18,9))
sns.boxplot(x = 'season', y = 'cnt', data = data_graphic, ax = ax1,)#Plot the demand according to season.
sns.boxplot(x = 'weathersit', y = 'cnt', data = data_graphic, ax = ax2)#Plot the demand according to weather.

On remarque que la demande est élevée en été et en printemps ce qui est normale à cause du climat et de la disponibilité des consommateurs ,de même pour la demande en temps clair .

In [ ]:
sns.catplot(x = 'hr', y = 'cnt', data = data_graphic, kind = 'box', aspect = 3) #Plot the demand according to the hour of the day

La valeur médiane est relativement plus élevée à 7h-8h et à 17h-18h. Cela peut être attribué aux utilisateurs réguliers des écoles et des bureaux à ces heures.

In [ ]:
data_graphic['mnth'] = data_graphic.mnth.map({1:'January', 2:'February', 3:'March', 4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}) # add the names of the months
plt.figure(figsize=(12,6))
sns.catplot(x = 'mnth', y = 'cnt', data = data_graphic, kind = 'box', aspect = 3) #Plot the demand according to the month

Il est évident que les gens ont tendance à ne pas louer des vélos pendant  saison d'hiver, car il n'est pas vraiment propice de faire du vélo à cette saison, c'est pourquoi les mois de janvier, février ont une demande relativement plus basse de vélos.


In [ ]:
#holiday
sns.factorplot(x='holiday',data=data_graphic,kind='count',size=5,aspect=1)
plt.show()

La plupart des points aberrants proviennent principalement des jours qui ne sont pas du vacances.


In [ ]:
#working days
sns.factorplot(x='workingday',data=data_graphic,kind='count',size=5,aspect=1)
plt.show()

La plupart des points aberrants proviennent principalement des jours de travail.



#build models


In [ ]:
#defining our input and output
X = data_train.drop('cnt',axis=1)
y = data_train['cnt']

In [ ]:
#display the scores of the mutual information to get rid of columns that will have no impact on the prediction
from sklearn.feature_selection import mutual_info_regression
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


discrete_features = X.dtypes == int
mi_scores = make_mi_scores(X, y, discrete_features)

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Scores de l'information mutuelles")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

"casual" et "registred" ne sont pas non plus pris en compte car il s'agit de variables de fuite par nature et elles doivent être abandonnées lors de la construction du modèle.

In [ ]:
X.drop(['casual','registered'], axis = 1, inplace = True)# get rid of columns that are not necessary for our prediction

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 0)#separate our data on training and testing data

In [ ]:
#linear regression 
from sklearn.linear_model import LinearRegression
# model fitting
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
#make predection
y_pred_ln = lr_reg.predict(X_test)
# checking the rmsle and the r2_score
print('r2_score:',r2_score(y_test,y_pred_ln))
print('rmsle:',mean_squared_log_error(y_test,y_pred_ln))


In [ ]:
#AdaBoostRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score, mean_squared_log_error
#fitting model AAdaBoostRegressor
abr = AdaBoostRegressor(random_state=42)
abr.fit( X_train , y_train )
y_pred= abr.predict(X_test)
# checking the rmsle and the r2_score
print('rmsle:',mean_squared_log_error(y_test,y_pred))
print('r2_score:',r2_score(y_test,y_pred))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# model fitting
model_rf = RandomForestRegressor(random_state=42)
model_rf.fit( X_train , y_train )
# Prediction
y_pred = model_rf.predict(X_test)
# checking the rmsle and the r2_score
print('rmsle:',mean_squared_log_error(y_test,y_pred))
print('r2_score:',r2_score(y_test,y_pred))

In [ ]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(n_estimators = 1000)
xgb_reg.fit(X_train, y_train)
y_pred_xg = xgb_reg.predict(X_test)

# checking the rmsle and the r2_score
print('rmsle:',mean_squared_log_error(y_test,y_pred_xg))
print('r2_score:',r2_score(y_test,y_pred_xg))

In [ ]:
#KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
model_knn = KNeighborsRegressor()
#fitting our model
model_knn.fit( X_train , y_train )
#prediction
y_pred_knn = model_knn.predict(X_test)
# checking the rmsle and the r2_score
print('rmsle:',mean_squared_log_error(y_test,y_pred_knn))
print('r2_score:',r2_score(y_test,y_pred_knn))

Notez que les modèles : régression linéaire, Xgboost ne sont pas bons pour notre prédiction. Le problème est que ces modèles prédisent des valeurs négatives pour la cible, ce qui n'est pas possible.

pour les trois modèls qu'ils reste (KNeighborsRegressor,AdaBoostRegressor,RandomForestRegressor) on remarque que le RandomForestRegressor donne le plus petit rmsle et le meilleur r2_score .pour cela  je l'utiliserai pour faire des prédictions. mais cette fois avec une optimisation des parametrs 


In [ ]:
# parametrs optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, random_state=42,shuffle=True)

params_dict = {'n_estimators':[100, 200, 300, 1000],'n_jobs':[-1],'max_features':["auto",'sqrt','log2']}
rf_reg = GridSearchCV(estimator= model_rf,param_grid = params_dict,scoring = 'neg_mean_squared_log_error',cv = kfold)
rf_reg.fit(X_train,y_train)
# checking the best parameter
rf_reg.best_params_

In [ ]:
#RandomForestRegressor with obtimized parametrs
from sklearn.ensemble import RandomForestRegressor
# model fitting
model_rf = RandomForestRegressor(max_features = 'auto', n_estimators= 400, n_jobs= -1,random_state=42)
model_rf.fit( X_train , y_train )
# Prediction
y_pred = model_rf.predict(X_test)
# checking the rmsle and the r2_score
print('rmsle:',mean_squared_log_error(y_test,y_pred))
print('r2_score:',r2_score(y_test,y_pred))


l'obtimisation des parametres n'a pas impacté le score méme si elle a pris assez du temps pour excuter le programme